In [1]:
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm
from scipy.integrate import simps
from numpy import trapz

from angstrom import Molecule
from wham2d import *
from dctst import *
from wham_plot import *

%matplotlib inline

## WHAM 2-D
Analyze WHAM-2D data.

In [ ]:
SCAN = 'DC_Cu110'
SCANDIR = '/home/kutay/Documents/git/Nanocar/surface-scan'
DATADIR = os.path.join(SCANDIR, 'analysis', 'data')
WHAMDIR = os.path.join(SCANDIR, 'analysis', 'wham', 'run2d')
HIST = dict(x=(21, 26), y=(22, 26))

In [ ]:
outfile = os.path.join(WHAMDIR, SCAN, 'wham-2d.out')
DATA = read_wham_out_list(outfile)

### Plot energy barriers

In [ ]:
fplot = []
for i, d in enumerate(DATA):
    fplot.append(dict(x=d['y'], y=d['free'], xlabel='y (Å)', ylabel='Free energy (kcal/mol)', title='%s | %i : %.2f' % (SCAN, i, d['x'][0]), xlim=HIST['y']))

In [ ]:
plt_file = os.path.join(SCANDIR, 'analysis', 'wham', 'plots', '%s-barriers.png' % SCAN)
subplot(wham_plot, fplot, width=6, dpi=300, nrow=5, save=None)

### Calculate Boltzmann average activation energy

In [ ]:
fq_star, fq_area = [], []
for i, d in enumerate(DATA):
    Fd = np.array(d['free'])
    Fd[np.isinf(Fd)] = 0
    if Fd.sum() != 0.0:
        fq_star.append(max(Fd))
        fq_area.append(simps(Fd, dx=0.1))
        # d['Ea'] = max(Fd)

fq_star_avg = calculate_boltzmann_average(fq_star, temperature=200)
fq_area_avg = calculate_boltzmann_average(fq_area, temperature=200)
print('Min F*: %.3f | Avg. F*: %.3f | Avg. F_area: %.3f' % (min(fq_star), fq_star_avg, fq_area_avg))

### Calculate rate and diffusion coefficient

$k_{A \rightarrow B} = \kappa {\sqrt{ \dfrac{1}{2 \pi M \beta}}} \dfrac{ e^{- \beta F(q^*)} } { \int e^{- \beta F(q)}dq } \ \ \rightarrow \ \  k_{A \rightarrow B} = \kappa \times 36.377 \sqrt{\dfrac{T}{M}} \dfrac{ e^{- \beta F(q^*)} } { \int e^{- \beta F(q)}dq }  \Big(\dfrac{1}{s}\Big)$

$D_s = \dfrac{1}{2n} \times k_{A \rightarrow B} \times \lambda^2$

In [ ]:
mol = Molecule(read='/home/kutay/Documents/git/Nanocar/molecules/surface-diffusion/opt/%s.xyz' % SCAN.split('_')[0])
mw = mol.get_molecular_weight()       # g / mol

In [ ]:
k_wham = calculate_rate(fq_star_avg, fq_area_avg, mw)
d_wham = calculate_diffusion(k_wham, lamda=1)
print('k: %.2e 1/s | D: %.2e cm2/s' % (k_wham, d_wham))

## Read all SCANs

In [ ]:
ALL_DATA = {}
for SCAN in os.listdir(WHAMDIR):
    outfile = os.path.join(WHAMDIR, SCAN, 'wham-2d.out')
    DATA = read_wham_out_list(outfile)[2:22]  # Limit xrange to 21.5 - 25.3
    
    fq_star, fq_area, x_wham, scan_ids = [], [], [], []
    for i, d in enumerate(DATA):
        Fd = np.array(d['free'][7:36])  # Limit yrange to 22.75 - 25.55
        Fd[np.isinf(Fd)] = 0
        if Fd.sum() != 0.0:
            fq_star.append(max(Fd))
            fq_area.append(simps(Fd, dx=0.1))
            x_wham.append(d['x'][0])
            scan_ids.append(i)

    fq_star_avg = calculate_boltzmann_average(fq_star, temperature=200)
    # Some barriers are discontinous causing wrong area calculations!!!
    # fq_area_avg = calculate_boltzmann_average(fq_area, temperature=200)
    fq_area_avg = fq_area[np.argsort(fq_star)[0]]
    f_min = DATA[scan_ids[np.argsort(fq_star)[0]]]['free'][7:36]  # Energy barrier with smallest F*
    f_min_y = DATA[scan_ids[np.argsort(fq_star)[0]]]['y'][7:36]
    
    mol = Molecule(read='/home/kutay/Documents/git/Nanocar/molecules/surface-diffusion/opt/%s.xyz' % SCAN.split('_')[0])
    mw = mol.get_molecular_weight()       # g / mol
    
    k_wham = calculate_rate(fq_star_avg, fq_area_avg, mw)
    d_wham = calculate_diffusion(k_wham, lamda=1)
    
    ALL_DATA[SCAN] = {'f_star': fq_star_avg, 'f_area': fq_area_avg, 'x': x_wham, 'f_min': f_min, 'f_min_y': f_min_y, 'f_star_x': fq_star, 'k': k_wham, 'd': d_wham, 'mw': mw}

#    # Save yaml file for surface plot
#     scan_yaml = dict(x=[], y=[], z=[])
#     for d in DATA:
#         scan_yaml['x'] += d['x']
#         scan_yaml['y'] += d['y']
#         scan_yaml['z'] += d['free']
#     surf_data_file = os.path.join(SCANDIR, 'analysis', 'surface-plot', 'data', '%s-2d-data.yaml' % SCAN)
#     with open(surf_data_file, 'w') as yf:
#         yaml.dump(scan_yaml, yf)
        
   # Save energy barriers
    fplot = []
    for i, d in enumerate(DATA):
        fplot.append(dict(x=d['y'][7:36], y=d['free'][7:36], xlabel='y (Å)', ylabel='Free energy (kcal/mol)', title='%s | %i : %.2f' % (SCAN, i, d['x'][0]), xlim=HIST['y']))
    plt_file = os.path.join(SCANDIR, 'analysis', 'wham', 'plots', '2D', '%s-barriers.png' % SCAN)
    subplot(wham_plot, fplot, width=6, dpi=300, nrow=5, save=plt_file)

### Plot minimum energy barrier for each molecule

In [ ]:
fig = plt.figure(figsize=(12, 9), dpi=100)
fig.subplots_adjust(hspace=.5, wspace=.25)

for i, scan in enumerate(ALL_DATA, start=1):
    ax = fig.add_subplot(3, 3, i)
    fmin = np.array(ALL_DATA[scan]['f_min'])
    fmin[np.isinf(fmin)] = 0
    ax.plot(ALL_DATA[scan]['f_min_y'], fmin, '-o', c='xkcd:crimson', lw=1.5, markersize=2)
    # ax.text(23, max(fmin) * 0.95, 'F(q)*: %.2f' % ALL_DATA[scan]['f_star'], color='k')
    ax.text(23, max(fmin) * 0.95, 'F(q)dq: %.2f' % ALL_DATA[scan]['f_area'], color='k')
    ax.set_title('%s' % scan)

In [ ]:
fig = plt.figure(figsize=(12, 9), dpi=100)
fig.subplots_adjust(hspace=.5, wspace=.25)

for i, scan in enumerate(ALL_DATA, start=1):
    ax = fig.add_subplot(3, 3, i)
    ax.plot(ALL_DATA[scan]['x'], ALL_DATA[scan]['f_star_x'], '-o', c='xkcd:crimson', lw=1.5, markersize=2)
    # ax.plot(ALL_DATA[scan]['x_g'], ALL_DATA[scan]['f_g'], '-', c='xkcd:blue', lw=1.5, markersize=2)
    # ax.text(22, max(ALL_DATA[scan]['f']) * 0.95, 'W: %.2f' % ALL_DATA[scan]['f_area'], color='xkcd:crimson')
    # ax.text(22, max(ALL_DATA[scan]['f_g']) * 0.85, 'G: %.2f' % ALL_DATA[scan]['f_area_g'], color='xkcd:blue')
    ax.set_title('%s' % scan)

In [ ]:
mw_all = [ALL_DATA[i]['mw'] for i in ALL_DATA]
mw_order = np.argsort(mw_all)
d_all = [ALL_DATA[i]['d'] for i in ALL_DATA]
k_all = [ALL_DATA[i]['k'] for i in ALL_DATA]
f_star_all = [ALL_DATA[i]['f_star'] for i in ALL_DATA]
f_area_all = [ALL_DATA[i]['f_area'] for i in ALL_DATA]
molecules = [d.split('_')[0] for d in ALL_DATA]

In [ ]:
f_star_exp = {'DC': 0.74, 'HtBDC': 0.57, 'PVBA': 0.83, 'C60': 1.40, 'DNHD': 0.195, 'TPEE': 0.175, 'BtPHD': 0.325, 'VL': 0, 'PCARBORANE': 0}
f_star_exp = [f_star_exp[i] * 23.061 for i in molecules]

In [ ]:
fig = plt.figure(figsize=(20, 3), dpi=100)
fig.subplots_adjust(hspace=.5, wspace=.25)

ax = fig.add_subplot(1, 4, 1)
ax.scatter(mw_all, d_all)
ax.set_title('D')
ax.set_ylim(0, 1e-14)
for i, d in enumerate(ALL_DATA):
    ax.text(mw_all[i], d_all[i], molecules[i])

ax = fig.add_subplot(1, 4, 2)
ax.scatter(mw_all, k_all)
ax.set_title('k')

ax = fig.add_subplot(1, 4, 3)
ax.scatter(mw_all, f_star_all)
ax.scatter(mw_all, f_star_exp)
ax.set_title('F(q)*')
ax.set_ylim(0, 35)
for i, d in enumerate(ALL_DATA):
    ax.text(mw_all[i],f_star_all[i], d.split('_')[0])

ax = fig.add_subplot(1, 4, 4)
ax.scatter(mw_all, f_area_all)
ax.set_title('F(q)dq')
# plt.ylim(0, 1e-14)
# plt.ylabel('Diffusion coefficient (cm2 / s)')
# plt.xlabel('Molecular weight (g / mol)')
# plt.legend(['WHAM', 'Gauss'])

### Save data for surface plot

In [ ]:
import yaml

scan_yaml = {'x': data['x'], 'y': data['y'], 'z': data['free']}
SURFDIR = os.path.join(SCANDIR, 'analysis', 'surface-plot')
with open(os.path.join(SURFDIR, '%s-2d-data.yaml' % SCAN), 'w') as yf:
    yaml.dump(scan_yaml, yf)